In [1]:
!pip install nested_dict dicthash

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import os

from multiarea_model import MultiAreaModel

# Down-scaled model

Neurons and indegrees are both scaled down to 1%.
Can usually be simulated on a local machine.

In [3]:
scale_down_to = 0.005

**Warning: This will not yield reasonable dynamical results from the
network and is only meant to demonstrate the simulation workflow.**

In [4]:
d = {}
conn_params = {'replace_non_simulated_areas': 'het_poisson_stat',
               'g': -11.,
               
               'K_stable': 'K_stable.npy',
               'fac_nu_ext_TH': 1.2,
               'fac_nu_ext_5E': 1.125,
               'fac_nu_ext_6E': 1.41666667,
               'av_indegree_V1': 3950.}
input_params = {'rate_ext': 10.}
neuron_params = {'V0_mean': -150.,
                 'V0_sd': 50.}
network_params = {'N_scaling': scale_down_to,
                  'K_scaling': scale_down_to,
                  'fullscale_rates': 'tests/fullscale_rates.json',
                  'input_params': input_params,
                  'connection_params': conn_params,
                  'neuron_params': neuron_params}

sim_params = {'t_sim': 2000.,
              'num_processes': 1,
              'local_num_threads': 1,
              'recording_dict': {'record_vm': False}}

theory_params = {'dt': 0.1}

In [5]:
M = MultiAreaModel(network_params, simulation=True,
                   sim_spec=sim_params,
                   theory=True,
                   theory_spec=theory_params)
p, r = M.theory.integrate_siegert()
print("Mean-field theory predicts an average "
      "rate of {0:.3f} spikes/s across all populations.".format(np.mean(r[:, -1])))

Initializing network from dictionary.
RAND_DATA_LABEL 1349
Model data base path: /opt/app-root/src/multi-area-model/multiarea_model/data_multiarea


/srv/test-build/spack/opt/spack/linux-centos7-broadwell/gcc-10.3.0/py-numpy-1.21.0-v2re3aybktzrk5wo5kutbfg4bea6oag3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning:Mean of empty slice.
/srv/test-build/spack/opt/spack/linux-centos7-broadwell/gcc-10.3.0/py-numpy-1.21.0-v2re3aybktzrk5wo5kutbfg4bea6oag3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning:invalid value encountered in double_scalars


No R installation, taking hard-coded SLN fit parameters.


Customized parameters
--------------------
{'K_scaling': 0.005,
 'N_scaling': 0.005,
 'connection_params': {'K_stable': 'K_stable.npy',
                       'av_indegree_V1': 3950.0,
                       'fac_nu_ext_5E': 1.125,
                       'fac_nu_ext_6E': 1.41666667,
                       'fac_nu_ext_TH': 1.2,
                       'g': -11.0,
                       'replace_non_simulated_areas': 'het_poisson_stat'},
 'fullscale_rates': 'tests/fullscale_rates.json',
 'input_params': {'rate_ext': 10.0},
 'neuron_params': {'V0_mean': -150.0, 'V0_sd': 50.0}}


/opt/app-root/src/.local/lib/python3.8/site-packages/dicthash/dicthash.py:47: UserWarning:Float too small for safe conversion tointeger. Rounding down to zero.


Simulation label: 3262d976c50ea3bf4743174127fdd9a8
Copied files.
Initialized simulation class.
Iteration: 0
Mean-field theory predicts an average rate of 29.588 spikes/s across all populations.


In [6]:
M.simulation.simulate()

Prepared simulation in 0.00 seconds.
Rank 0: created area V1 with 5587 local nodes
Memory after V1 : 1401.15 MB
Rank 0: created area V2 with 12359 local nodes
Memory after V2 : 1428.81 MB
Rank 0: created area VP with 19107 local nodes
Memory after VP : 1459.11 MB
Rank 0: created area V3 with 26024 local nodes
Memory after V3 : 1488.66 MB
Rank 0: created area V3A with 30934 local nodes
Memory after V3A : 1509.58 MB
Rank 0: created area MT with 36731 local nodes
Memory after MT : 1536.38 MB
Rank 0: created area V4t with 42326 local nodes
Memory after V4t : 1561.34 MB
Rank 0: created area V4 with 48809 local nodes
Memory after V4 : 1590.08 MB
Rank 0: created area VOT with 54420 local nodes
Memory after VOT : 1616.06 MB
Rank 0: created area MSTd with 59652 local nodes
Memory after MSTd : 1642.70 MB
Rank 0: created area PIP with 64596 local nodes
Memory after PIP : 1667.27 MB
Rank 0: created area PO with 69540 local nodes
Memory after PO : 1691.75 MB
Rank 0: created area DP with 74306 local

In [10]:
data = np.loadtxt(M.simulation.label + "-spikes-1-0.dat", skiprows=3)

In [11]:
data

array([[5.0000e+01, 1.0000e-01],
       [8.2000e+01, 1.0000e-01],
       [3.3700e+02, 1.0000e-01],
       ...,
       [1.8629e+04, 2.0000e+03],
       [2.0259e+04, 2.0000e+03],
       [2.0265e+04, 2.0000e+03]])

In [29]:
tsteps, spikecount = np.unique(data[:,1], return_counts=True)

In [35]:
import matplotlib.pyplot as plt

In [36]:
fig, ax = plt.subplots()
ax.plot(tsteps, spikecount)

<IPython.core.display.Javascript object>

In [37]:
plt.show()

In [54]:
len(spikecount), np.average(spikecount)

(19979, 36.330497021872965)

In [52]:
np.average(spikecount[2000:])

36.55876300127927